In [ ]:
%matplotlib widget
import matplotlib.pylab as plt
import numpy as np
import sympy as sp
from bmcs_utils.api import Cymbol
sp.init_printing()

In [ ]:
sig_c1 = Cymbol(r'\sigma_{c1}', codename='sig_c1')
sig_c2 = Cymbol(r'\sigma_{c2}', codename='sig_c2')
sig_c3 = Cymbol(r'\sigma_{c3}', codename='sig_c3')
eps_cu1 = Cymbol(r'\varepsilon_{\mathrm{cu1}}', codename='eps_cu1')
b = Cymbol(r'b', codename='b')
d = Cymbol(r'd', codename='d')
A_f = Cymbol(r'A_{\mathrm{f}}', codename='A_f')
f_fu = Cymbol(r'f_{\mathrm{fu}}', codename='f_fu')
E_f = Cymbol(r'E_{\mathrm{f}}', codename='E_f')
rho_f = Cymbol(r'\rho_{\mathrm{f}}', codename='rho_f')
sigma_f = Cymbol(r'\sigma_{\mathrm{f}}', codename='sigma_f')

eps_cy = Cymbol(r'\varepsilon_{\mathrm{cy}}', codename='eps_cy')
eps_cu = Cymbol(r'\varepsilon_{\mathrm{cu}}', codename='eps_cu')
eps_top = Cymbol(r'\varepsilon_{\mathrm{top}}', codename='eps_top')
eps_bot = Cymbol(r'\varepsilon_{\mathrm{bot}}', codename='eps_bot')
E_cc = Cymbol(r'E_{\mathrm{cc}}', codename='E_cc')
f_cm = Cymbol(r'f_{\mathrm{cm}}', codename='f_cm')
eps_1 = Cymbol(r'\varepsilon_{1}', codename='eps_1')
eta = Cymbol(r'\eta', codename='eta')
eta_cu = Cymbol(r'\eta_\mathrm{cu}', codename='eta_cu')
eta_top = Cymbol(r'\eta_\mathrm{top}', codename='eta_top')
z_top = Cymbol(r'z_\mathrm{top}', codename='z_top')
z = Cymbol(r'z', codename='z')
eta_int = Cymbol(r'\eta_\mathrm{int}', codename='eta_int')
k = Cymbol(r'k', codename='k')
kappa = Cymbol(r'\kappa', codename='kappa')


## Eurocode 2 function

In [ ]:
k_ = 1.05 * E_cc * eps_cy / f_cm
sig_c_EC2_ = sp.simplify(f_cm * (k * eta - eta**2)/(1 + eta*(k-2)) 
                     )
sig_c_EC2_

In [ ]:
m = Cymbol(r'm')
eta_0 = Cymbol(r'eta_0')
from sympy.stats import Weibull, density

# Create a Weibull random variable
X = Weibull('X', m, eta_0)

# Get the probability density function (pdf) of X
sig_c_weibull_ = f_cm * density(X)(eta)
sig_c_weibull_

In [ ]:
get_k = sp.lambdify((f_cm, E_cc, eps_cy), k_ )
get_sig_c_EC2 = sp.lambdify((eta, f_cm, eta_cu, k), sig_c_EC2_)
get_sig_c_sig_c_weibull = sp.lambdify((eta, f_cm, eta_0, m), sig_c_weibull_)

In [ ]:
_k = get_k(30, 28000, 0.002)
eta_range = np.linspace(-1.5, 0.8*_k, 2000)
fig, ax = plt.subplots(1,1)
sig_range = get_sig_c_EC2(eta_range, 30, 0.8*_k, _k)
#ax.plot(eta_range, sig_range)
sig_range_signoid = get_sig_c_sig_c_weibull(eta_range, 1, 1, 0.3)
ax.plot(eta_range, sig_range_signoid)


In [ ]:
with sp.assuming(sp.Q.is_true(f_cm > 0), 
                 sp.Q.is_true(E_cc > 0),
                 sp.Q.is_true(eta_cu > 0),
                 sp.Q.is_true(k > 1),
                 sp.Q.is_true(eta_int >= 0)):
    int_sig_c_eta_ = sp.simplify(
        sp.integrate(sig_c_EC2_, (eta, 0, eta))
    )
int_sig_c_eta_

In [ ]:
get_int_sig_c_eta = sp.lambdify((eta, f_cm, eta_cu, k, b),
                                 int_sig_c_eta_, modules='numpy')

In [ ]:
fig, ax = plt.subplots(1,1)
sig_range = get_int_sig_c_eta(eta_range, 30, 0.8*_k,  _k, 1)
ax.plot(eta_range, sig_range)
ax.plot([_k, _k],[0, np.max(sig_range)])

# Integral over compression zone

First express the relative compression strain as a linear function of the vertical coordinate $z$ starting at the neutral zone and ending at the top of the cross section

In [ ]:
eta_z_ = eta_top / z_top * z
eta_z_

In [ ]:
sig_c_z_ = sig_c_eta_.subs(eta, eta_z_)

In [ ]:
with sp.assuming(sp.Q.is_true(f_cm > 0), 
                 sp.Q.is_true(E_cc > 0),
                 sp.Q.is_true(eta_cu > 0),
                 sp.Q.is_true(eta_top >= 0),
                 sp.Q.is_true(b > 0),
                 sp.Q.is_true(k > 1),
                 sp.Q.is_true(z_top > 0)):
    F_c_eta_top_ = sp.simplify(
        sp.integrate(sig_c_z_ * b, (z, 0, z_top))
    )
F_c_eta_top_

## Find the position of compression zone stress resultant

In [ ]:
with sp.assuming(sp.Q.is_true(f_cm > 0), 
                 sp.Q.is_true(E_cc > 0),
                 sp.Q.is_true(eta_cu > 0),
                 sp.Q.is_true(eta_top >= 0),
                 sp.Q.is_true(b > 0),
                 sp.Q.is_true(k > 1),
                 sp.Q.is_true(z_top > 0)):
    M_c_eta_top_ = sp.simplify(
        sp.integrate(sig_c_z_ * b * z, (z, 0, z_top))
    )

In [ ]:
z_cg_eta_top_ = sp.simplify(M_c_eta_top_ / F_c_eta_top_)

In [ ]:
z_cg_eta_top_

In [ ]:
get_F_c_eta_top = sp.lambdify((eta_top, z_top, f_cm, eta_cu, k, b),
                                 F_c_eta_top_)
get_z_cg_eta_top = sp.lambdify((eta_top, z_top, f_cm, eta_cu, k, b),
                                 z_cg_eta_top_)

In [ ]:
fig, ax = plt.subplots(1,1)
ax_z = ax.twinx()
eta_range = np.linspace(0.001, 0.8*_k, 100)
int_sig_range = get_F_c_eta_top(eta_range, 10, 30, 0.8*_k,  _k, 10)
ax.plot(eta_range, int_sig_range)
ax.plot([_k, _k],[0, np.max(sig_range)])
z_cg_range = get_z_cg_eta_top(eta_range, 10, 30, 0.8*_k,  _k, 10)
ax_z.plot(eta_range, z_cg_range);

Is the maximum of $F_\mathrm{c}(\eta)$ always at $\eta_\mathrm{cu}$?

In [ ]:
z_top_ = sp.solve(sp.Eq((eps_cu + f_fu / E_f)/d, eps_cu/z_top), z_top)[0]
z_top_

In [ ]:
F_cu_ = sp.simplify(
    F_c_eta_top_.subs(eta_top, eta_cu).subs(eta_cu, eps_cu/eps_cy).subs(k,k_).subs(z_top, z_top_))
F_cu_

In [ ]:
A_f_ = rho_f * b * d
F_fu_ = A_f_ * f_fu
F_fu_


In [ ]:
rho_f_ = sp.solve(sp.Eq(F_cu_, F_fu_), rho_f)[0]

The inner lever arm is given as the difference 
$$
 z_\mathrm{inner} = d - z_\mathrm{top} + z_\mathrm{cg}
$$
Then, the bending moment with respect to the reinforcement layer is given as
$$
 M_\mathrm{r} = F_\mathrm{c} z_\mathrm{inner}
$$

In [ ]:
z_cg_cu_ = sp.simplify(
    z_cg_eta_top_.subs(eta_top, eta_cu).subs(eta_cu, eps_cu/eps_cy).subs(k,k_).subs(z_top, z_top_))
z_cg_cu_

In [ ]:
z_inner_ = sp.simplify(d - z_top_ + z_cg_cu_)

In [ ]:
M_cu_ = F_fu_.subs(rho_f, rho_f_) * z_inner_

In [ ]:
M_wo_b_ = sp.collect(M_cu_, d*d)
M_wo_b_

In [ ]:
M_E = Cymbol(r'M_\mathrm{E}', codename='M_E')

In [ ]:
# sp.solve(sp.Eq(M_E, M_cu_), d)

The curvature is given as
$$
 \kappa = \frac{\varepsilon_\mathrm{top}}{z_\mathrm{top}}
$$
Therefore, the height of the compression zone reads
$$
z_\mathrm{top} = \frac{\varepsilon_\mathrm{top}}{\kappa} 
$$

Given the strain $\varepsilon_\mathrm{top}$ and curvature 
$$
 \kappa = \frac{\varepsilon_\mathrm{top} + \varepsilon_\mathrm{bot}}{d}
$$

In [ ]:
eps_bot_ = sp.solve(sp.Eq(kappa, (eps_top + eps_bot) / d), eps_bot)[0]
eps_bot_

In [ ]:
F_r_eps_top_kappa_ = A_f * E_f * eps_bot_ 
F_r_eps_top_kappa_

In [ ]:
# defining symbols
x, y = sp.symbols('x y')

# defining a polynomial
poly = x*y + x + y + 1

# collecting terms
collected_poly = sp.collect(poly, x)

# extracting coefficient
collected_poly.coeff(x)


In [ ]:
collected_poly